## img 2장으로 파노라마 연출하기

In [ ]:
import cv2, numpy as np
import matplotlib.pyplot as plt

#파노라마로 만들 이미지 불러오기
img1 = cv2.imread('img/restaurant1.jpg')
img2 = cv2.imread('img/restaurant2.jpg')

# 두 사진의 높이와 너비
h1, w1 = img1.shape[:2]
h2, w2 = img2.shape[:2]

#그레이 스케일링
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#ORB 검출기 생성
descriptor = cv2.ORB_create()
(kp1, desc1) = descriptor.detectAndCompute(gray1, None)
(kp2, desc2) = descriptor.detectAndCompute(gray2, None)

# print(len(kp1))
# print(len(kp2))
# print(desc1.shape)
# print(desc2.shape)

#BRF 매칭기 생성
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)

#매칭
matches = matcher.knnMatch(desc1, desc2, 2)

#이웃 거리의 95%로 좋은 매칭점 추출
ratio = 0.95
good_matches = [first for first,second in matches if first.distance<second.distance*ratio]
print('matches:%d/%d' %(len(good_matches), len(matches)))

#img1과 img2를 나란히 두고 매칭점 그려서 출력
res = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
cv2.imshow('BFMatcher + SIFT', res)
cv2.waitKey()
cv2.destroyAllWindows()

#좋은 매칭점으로 원본과 대상 영상의 좌표 구하기
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ])
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ])
#원근 변환 행렬 구하기
mtrx, mask = cv2.findHomography( dst_pts,src_pts, cv2.RANSAC, 4.0)
# print(len(mtrx))
# print(len(mask))

#img2에 mtrx를 적용한 뒤 panorama에 담기
panorama = cv2.warpPerspective(img2, mtrx, (w2+w2, h2))

#왼쪽 절반에 img1 담기
panorama[0:h1, 0:w1] = img1

cv2.imshow('BFMatcher + SIFT', panorama)
cv2.waitKey()
cv2.destroyAllWindows()

matches:295/500
